In [1]:
from app.dotenv import base_dir, data_dir
from app.model.model import  Model
import os

In [2]:
print('dotenv params:', {os.getenv('PARAM1')})
print('base_dir:', base_dir)
print('data_dir:', data_dir)

model = Model()
print('model:', model)



dotenv params: {'PARAM1_value'}
base_dir: c:\Users\yili0901\Downloads\code\ivrit_ai\app
data_dir: c:\Users\yili0901\Downloads\code\ivrit_ai\data
model: loaded Model


In [3]:
dataset_name1 = 'C:/Users/yili0901/Downloads/code/ivrit_ai/data/audio-transcripts'
dataset_name2 = 'C:/Users/yili0901/Downloads/code/ivrit_ai/data/whisper-training'
# dataset_name2 = dataset_name1
r=5

In [4]:
from datasets import load_dataset, Dataset, DatasetDict,concatenate_datasets

# Load the first Huggingface dataset
dataset1 = load_dataset(dataset_name1)
# dataset1 = dataset1['train']  # Adjust according to your dataset split
dataset1 = concatenate_datasets([dataset1['train']])

print(dataset1)
print('_'*50)
# Load the second Huggingface dataset
dataset2 = load_dataset(dataset_name2)
# dataset2 = dataset2['train']  # Adjust according to your dataset split
dataset2 = concatenate_datasets([dataset2['train'], dataset2['test']])
# dataset2 = concatenate_datasets([dataset2['train']])

print(dataset2)



c:\Users\yili0901\Downloads\code\ivrit_ai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['source', 'episode', 'uuid', 'text', 'attrs'],
    num_rows: 2183042
})
__________________________________________________
Dataset({
    features: ['source', 'episode', 'uuid', 'text', 'attrs'],
    num_rows: 2183042
})


In [16]:
import random
# Convert UUIDs to sets for comparison
set_uuids1 = set(dataset1['uuid'])
set_uuids2 = set(dataset2['uuid'])

# Compare the UUIDs
common_uuids = set_uuids1 & set_uuids2
# unique_to_dataset1 = set_uuids1 - set_uuids2
# unique_to_dataset2 = set_uuids2 - set_uuids1


# sub sample
common_uuids =set(list(common_uuids)[:10])
common_uuids

{'AnotherPodcastForStartups/2023.01.18 [מובילי חדשנות] כיצד הבינה המלאכותית תשנה את חיינו/325',
 'Geekonomy/2016.03.14 גיקונומי #42 - אורי כץ/991',
 'Geekonomy/2019.06.16 פרק #262 - אהרון אהרון ורשות החדשנות/0',
 'Geekonomy/2020.06.07 פרק #334 - יגאל ליברנט והמורכבות האנושית/758',
 'Geekonomy/2021.12.29 פרק #496 - אור דיין מדווש בעליה/924',
 'Geekonomy/2022.08.02 פרק #589 - יואב לוי וסייבר בתעשיית הרכב/44',
 'Geekonomy/2023.02.14 פרק #681 - הטור האישי של רענן שקד/44',
 'SmallBigHistory/2019.03.24 הולכים לבחירות #3 - גולדה מאיר/320',
 'SmallBigHistory/2020.06.10 #108 - דני אורבך יוצא להתנקש בהיטלר/334',
 'Tziun3/2021.12.20 פרק #271 – מסכמים סיבוב בגביע/446'}

In [17]:
# Create a mapping dictionary from uuid to the desired fields in dataset1
fields_to_add = dataset1.column_names  # Adjust this list based on your requirements
uuid_mapping1 = {row['uuid']: {field+'1': row[field] for field in fields_to_add} for row in dataset1 if row['uuid'] in common_uuids}
type(uuid_mapping1)

# Create a mapping dictionary from uuid to the desired fields in dataset1
fields_to_add = dataset2.column_names  # Adjust this list based on your requirements
uuid_mapping2 = {row['uuid']: {field+'2': row[field] for field in fields_to_add} for row in dataset2 if row['uuid'] in common_uuids}
type(uuid_mapping2)


# Define a function to update rows in dataset2 based on the uuid_mapping
def update_row(uuid,row):
    # uuid = row['uuid']
    if uuid in uuid_mapping1:
        for field, value in uuid_mapping1[uuid].items():
            row[field] = value
    return row
updated_uuid_mapping = {uuid: update_row(uuid, row.copy()) for uuid, row in uuid_mapping2.items()}
updated_dataset = Dataset.from_dict({key: [row[key] for row in updated_uuid_mapping.values()] for key in updated_uuid_mapping[list(updated_uuid_mapping.keys())[0]].keys()})


In [18]:


uuid = list(updated_uuid_mapping.keys())[0]
print(updated_uuid_mapping[uuid].keys())
print(uuid_mapping1[uuid].keys())
print(uuid_mapping2[uuid].keys())

dict_keys(['source2', 'episode2', 'uuid2', 'text2', 'attrs2', 'source1', 'episode1', 'uuid1', 'text1', 'attrs1'])
dict_keys(['source1', 'episode1', 'uuid1', 'text1', 'attrs1'])
dict_keys(['source2', 'episode2', 'uuid2', 'text2', 'attrs2'])


In [21]:
import pandas as pd

df = pd.DataFrame(updated_dataset)

# Display the DataFrame
# print(df.head())
df.columns

Index(['source2', 'episode2', 'uuid2', 'text2', 'attrs2', 'source1',
       'episode1', 'uuid1', 'text1', 'attrs1'],
      dtype='object')